# 라이브러리 임포트

In [12]:
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver

import time
import pandas as pd
import numpy as np
import random


# 파일 위치 설정

In [ ]:
# 크롬 드라이버 제어
driver = webdriver.Chrome()